In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-3.3B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

In [ ]:
# pip install openai

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#pip install paraphraser

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

In [ ]:
pip install rouge

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.tag import pos_tag
import time
from rouge import Rouge
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import files

translator = pipeline("translation", model="facebook/nllb-200-3.3B")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

def translate_text(text, target_language='en'):
    try:
        print(target_language)
        time.sleep(1)
        translation = translator(text, src_lang='en', tgt_lang=target_language)
        return translation[0]['translation_text']
    except Exception as e:
        print(f"Translation failed: {e}")
        return None

def calculate_rouge_score(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]['rouge-l']['f']

def generate_paraphrases(sentence, num_paraphrases=1):
    paraphrases = set()
    tokens = word_tokenize(sentence)
    tagged_tokens = pos_tag(tokens)

    pos_groups = {}
    for token, tag in tagged_tokens:
        if tag in pos_groups:
            pos_groups[tag].append(token)
        else:
            pos_groups[tag] = [token]

    for tag, tokens_group in pos_groups.items():
        if tag.startswith('NN') or tag.startswith('VB') or tag.startswith('JJ'):
            synonyms_group = set()
            for token in tokens_group:
                for syn in wordnet.synsets(token):
                    for lemma in syn.lemmas():
                        synonym = lemma.name().replace("_", " ")
                        if lemma.name() != token and wordnet.synsets(synonym):
                            similarity_threshold = 0.7
                            if syn.path_similarity(wordnet.synsets(token)[0]) is not None and syn.path_similarity(wordnet.synsets(synonym)[0]) is not None:
                                if syn.path_similarity(wordnet.synsets(token)[0]) > similarity_threshold and syn.path_similarity(wordnet.synsets(synonym)[0]) > similarity_threshold:
                                    synonyms_group.add(synonym)

            if synonyms_group:
                for _ in range(min(num_paraphrases, len(synonyms_group))):
                    paraphrase_tokens = tokens[:]
                    for original_token, synonym in zip(tokens_group, synonyms_group):
                        for i, token in enumerate(p for p in paraphrase_tokens if p == original_token):
                            paraphrase_tokens[paraphrase_tokens.index(token)] = synonym
                    paraphrase = ' '.join(paraphrase_tokens)
                    if paraphrase != sentence:
                        paraphrases.add(paraphrase)
                    if len(paraphrases) >= num_paraphrases:
                        break
        if len(paraphrases) >= num_paraphrases:
            break
    return list(paraphrases)[:num_paraphrases]

def translate_csv(input_csv, output_csv, target_languages):
    df = pd.read_csv(input_csv)

    rows = []

    for index, row in df.iterrows():
        for lang in target_languages:
            translated_prompt = translate_text(row['English_Prompt'], lang)
            back_translated_prompt = translate_text(translated_prompt)
            rouge_score = calculate_rouge_score(row['English_Prompt'], back_translated_prompt)
            rows.append([row['English_Prompt'], translated_prompt, back_translated_prompt, rouge_score, lang, row['Expected_Long_Term'], row['Expected_Short_Term']])

    translated_df = pd.DataFrame(rows, columns=['English_Prompt', 'Translated', 'Back_Translated', 'ROUGE_Score', 'Target_Language', 'Expected_Long_Term', 'Expected_Short_Term'])
    translated_df.to_csv(output_csv, index=False)
    files.download(output_csv)

if __name__ == "__main__":
    input_csv = "/content/long_term_vs_short_term_orientation.csv"
    output_csv = "output_longterm.csv"

    target_languages = [
        "en",  # English
        "de",  # German
        "ru",  # Russian
        "ja",  # Japanese
        "fr",  # French
        "it",  # Italian
        "zh-CN",  # Chinese
        "id",  # Indonesian
        "tr",  # Turkish
        "nl",  # Dutch
        "pl",  # Polish
        "fa",  # Persian
        "ko",  # Korean
        "cs",  # Czech
        "uk",  # Ukrainian
        "hu",  # Hungarian
        "el",  # Greek
        "ro",  # Romanian
        "sv",  # Swedish
        "he",  # Hebrew
        "da",  # Danish
        "th",  # Thai
        "fi",  # Finnish
        "bg",  # Bulgarian
        "kk",  # Kazakh
        "hy",  # Armenian
        "ka",  # Georgian
        "sq",  # Albanian
        "az",  # Azerbaijani
        "ms",  # Malay
        "mn",  # Mongolian
        "be",  # Belarusian
        "hi",  # Hindi
        "af",  # Afrikaans
        "is",  # Icelandic
        "si"   # Sinhala
    ]

    translate_csv(input_csv, output_csv, target_languages)



In [ ]:
# !pip install --upgrade googletrans==4.0.0-rc1
# from googletrans import Translator
# translator = Translator()
# text = 'My name is Julia'
# translation = translator.translate(text, dest='ru')
# translation.text